In [1]:
# AUTHOR: LI WAN | UNIVERSITY OF CAMBRIDGE

import os
import scipy.io as sio
import pandas as pd
import numpy as np
import time

# from numba import jit
# from collections import namedtuple
#maybe only keep 4 decimals for all the results  - do it later

## Input

In [2]:
# parameters
MaxITN = 5000         # max iteration times
Tol = 1e-6            # tolerance       ## 10^(-03) = 1.0000e-03 = 0.001, is this right? 
Status_Mode = 0       # 1: Calibdation mode; 0: Forecast mode
Status_EmpPred = 1    # 1: predict emp-residential location pair; 0: predict residential location only 
Status_HrentPred = 1  # 1: predict house rents; 0: Exogenous house rents
LLCoefIJ = np.array([[0.0,0.0]]) # log-linear transformation coef
D = 250               # number of working days
Lambda = np.array([[1.0,1.0]])

In [3]:
# vairables
LT = len(Lambda[0]) # labour type       ##size(Lambda,2): number of second dimension (variables/columns) - here, the result is 2. 

EmpSeCTot = np.array([[300,1]]) # total employment by socio-economic classification; input if Status_EmpPred == 1
                                # left-side (100) is high-income group; right-hand side (1) is low income group

EmpSeC = np.array([[100,1],   
                   [100,1],
                   [100,1]])    # employment by socio-economic classification; input if Status_EmpPred == 0
                                # from top to bottom - zone 1, 2, 3 respectively. Left (100) is high-income group; right-hand side (1) is low income group
                           
Time1 = np.array([[5,15,30],    # 5 means: time from living in zone 1 and work in zone 1; 15 means: living in zone 1 but work in zone 2
                  [15,5,15],    # the first 15 means: living in zone 2 but work in zone 1
                  [30,15,5]]) 
                                # travel time matrix (Unit: minute)

Time = np.repeat(Time1[None,...],LT,axis=0)   # Time = repmat(Time1, [1,1,LT])    # Time.shape (2,3,3) - this is right. Here, means 2 layers, 3 rows and 3 columns. = size (3,3,2) in Matlab  
Dist = Time # travel distance matrix (Unit: km)


HS = np.array([[1000],
               [1000],
               [1000]])       # housing floorspace - zone 1, 2, 3 respectively         # OR: np.array([1000,1000,1000])

BFS = np.array([[1000],
                [1000],
                [1000]])      # business floorspace

Hrent0 = np.array([[200],
                   [200],
                   [200]])    # unit house rent

Hrent = Hrent0


ZNum = len(HS)      # zone number    # size(HS,1): number of the first dimension (rows) - here, the result is 3

# # read mat file generated from python (carlibration mode)
# if os.path.isfile('ZAT(Python).mat'):
#     print('------------------- ZAT file exists - Load ZAT file -----------------')
    
#     ZAttrI = sio.loadmat('ZAT(Python).mat')['ZAttrI']
#     ZAttrIJ = sio.loadmat('ZAT(Python).mat')['ZAttrIJ']


# read the original mat file generated from matlab, need to change axis order (maybe different axix order issue)
if os.path.isfile('ZAT.mat'):
    print('------------------- ZAT file exists - Load ZAT file -----------------')
    matZAT = sio.loadmat('ZAT.mat')['ZAT']
    ZAT = matZAT[0,0]    # ZAT.dtype
    ZAttrI = np.moveaxis(ZAT['ZAttrI'], -1, 0)
    ZAttrIJ = np.moveaxis(ZAT['ZAttrIJ'], -1, 0)

else:
    print('-------------- ZAT file not exists - Replace with zeros -------------')
    ZAttrIJ = np.zeros((LT,ZNum,ZNum))   # == Matlab: zeros(ZNum,ZNum,LT).   Python: layers first, then rows*columns
    ZAttrI = np.zeros((LT,ZNum,ZNum))

------------------- ZAT file exists - Load ZAT file -----------------


In [4]:
# Data input for updating Hrent
Wage = np.array([[10000,10000],   
                 [10000,10000],
                 [10000,10000]])

HSExpShare = np.array([[0.2,0.2],   
                       [0.2,0.2],
                       [0.2,0.2]])


if Status_EmpPred == 1:
    EmpInput = EmpSeCTot
else:
    EmpInput = EmpSeC

## Function

In [5]:
from functions import ProbIJ_Mix, Update_Hrent, Calibrate_ZAttr

## Iteration - Calculate location choice probability

In [6]:
# 0.8737 seconds

start_time = time.time()

if Status_HrentPred == 1:
    print('--------------------------- Iteration starts ------------------------')
    
    for k in list(range(1,MaxITN+1)):
        
        if k == MaxITN:
            print('-------------------------- MaxITN reached --------------------------')
            break
        
        Output = ProbIJ_Mix(Status_EmpPred,D,LLCoefIJ,Lambda,EmpInput,Time,Dist,HS,BFS,Hrent0,ZAttrIJ,ZAttrI, LT,ZNum) #add LT,ZNum
        Hrent, Error = Update_Hrent(Output, LT,ZNum,Wage,HSExpShare,Hrent0,HS)
        
        if Error < Tol:
            print('--------------------- Hrent Converged at ITN = {} ------------------'.format(k))
            break
        else:
            Hrent0 = 1.0*Hrent + 0.0*Hrent0
            continue
    
else:
    print('--------------- Calculate location choice probability ---------------')
    Output = ProbIJ_Mix(Status_EmpPred,D,LLCoefIJ,Lambda,EmpInput,Time,Dist,HS,BFS,Hrent0,ZAttrIJ,ZAttrI, LT,ZNum)
    

if Status_Mode == 1:
    print('---------------------- ZATTR Calibration start ----------------------')
    ZAttrIJ,ZAttrI = Calibrate_ZAttr(D,LLCoefIJ,Lambda,Time,HS,BFS,Hrent, LT,ZNum)
    sio.savemat('ZAT(Python).mat', {'ZAttrIJ':ZAttrIJ, 'ZAttrI':ZAttrI})

print("Elapsed time is: %.4f seconds" % (time.time() - start_time)) 

--------------------------- Iteration starts ------------------------
--------------------- Hrent Converged at ITN = 1832 ------------------
Elapsed time is: 0.8041 seconds


In [7]:
Hrent

array([[208.03587586],
       [185.92768657],
       [208.03587586]])

In [8]:
Error

9.988365373550323e-07

In [9]:
Output['IJ']

array([[[5.92414094e+01, 2.96207047e+01, 1.48103523e+01],
        [1.54425112e+01, 6.17700447e+01, 1.54425112e+01],
        [1.48103523e+01, 2.96207047e+01, 5.92414094e+01]],

       [[1.97471365e-01, 9.87356823e-02, 4.93678412e-02],
        [5.14750373e-02, 2.05900149e-01, 5.14750373e-02],
        [4.93678412e-02, 9.87356823e-02, 1.97471365e-01]]])

## Write output files

In [10]:
Date = ['DATE: ',pd.Timestamp.today()]     # change format later - currently they're in 2 columns
Project = ['PROJECT NAME: ProbIJ_Model_Test']
Author = ['AUTHOR: LI WAN | UNIVERSITY OF CAMBRIDGE']
Precision = ['PRECISION: ',Tol]


if Status_Mode == 1:
    ModelMode = ['MODEL MODE: CALIBRATION']
else:
    ModelMode = ['MODEL MODE: FORECAST']

    
if Status_EmpPred == 1:
    EmpPredMode = ['EMPLOTMENT PREDICTION: ENABLED']
else:
    EmpPredMode = ['EMPLOTMENT PREDICTION: DISABLED']


if Status_HrentPred == 1:
    HrentPredMode = ['HOUSE RENTS PREDICTION: ENABLED'];
else:
    HrentPredMode = ['HOUSE RENTS PREDICTION: DISABLED'];


Metadata = [Project,Date,Author,Precision,ModelMode,EmpPredMode,HrentPredMode]
MetadataT = pd.DataFrame(data = Metadata)
#Matlab: Output.Metadata = MetadataT  #save in the output construct, check later. 

In [11]:
# 2d array to dataframe
df_ER = pd.DataFrame(Output['ER'], columns = pd.MultiIndex.from_tuples([('ER','Column_A'),('ER','Column_B')]))  # when checking the excel file, there is a empty gap between column name and content - do this later!!
df_EW = pd.DataFrame(Output['EW'], columns = pd.MultiIndex.from_tuples([('EW','Column_A'),('EW','Column_B')]))
T_EREW = pd.concat([df_ER, df_EW], axis=1)

df_JobOpp = pd.DataFrame(Output['JobOpp'], columns = pd.MultiIndex.from_tuples([('JobOpp','Column_A'),('JobOpp','Column_B')]))  # format gap - do this later
df_LabCat = pd.DataFrame(Output['LabCat'], columns = pd.MultiIndex.from_tuples([('LabCat','Column_A'),('LabCat','Column_B')]))
T_JobOppLatCat = pd.concat([df_JobOpp, df_LabCat], axis=1)

df_ACD = pd.DataFrame(Output['ACD'], columns = pd.MultiIndex.from_tuples([('ACD','Column_A'),('ACD','Column_B')]))  # format gap - do this later
df_ACT = pd.DataFrame(Output['ACT'], columns = pd.MultiIndex.from_tuples([('ACT','Column_A'),('ACT','Column_B')]))
T_Tran = pd.concat([df_ACD, df_ACT], axis=1)


# save 3d array to dataframe
names = ['dim3', 'dim_row', 'dim_column']

index_IJ = pd.MultiIndex.from_product([range(s)for s in Output['IJ'].shape], names=names)
df_IJ = pd.DataFrame({'IJ': Output['IJ'].flatten()}, index=index_IJ)['IJ']
df_IJ = df_IJ.unstack(level='dim_column')#.swaplevel().sort_index() 

index_ProbIJ = pd.MultiIndex.from_product([range(s)for s in Output['ProbIJ'].shape], names=names)
df_ProbIJ = pd.DataFrame({'ProbIJ': Output['ProbIJ'].flatten()}, index=index_ProbIJ)['ProbIJ']
df_ProbIJ = df_ProbIJ.unstack(level='dim_column')#.swaplevel().sort_index() 

index_ProbI = pd.MultiIndex.from_product([range(s)for s in Output['ProbI'].shape], names=names)
df_ProbI = pd.DataFrame({'ProbI': Output['ProbI'].flatten()}, index=index_ProbI)['ProbI']
df_ProbI = df_ProbI.unstack(level='dim_column')#.swaplevel().sort_index() 


# write to the excel file
Filename = pd.ExcelWriter('_Output_Summary(python).xlsx') #, engine='xlsxwriter'
MetadataT.to_excel(Filename, sheet_name='Metadata', index=False)
df_IJ.to_excel(Filename, sheet_name='Commuting_Flow')
df_IJ_all = pd.DataFrame(sum([Output['IJ'][l] for l in list(range(0,Output['IJ'].shape[0]))]))
df_IJ_all.to_excel(Filename, sheet_name='Commuting_Flow_All', index=False)
T_EREW.to_excel(Filename, sheet_name='ER_EW')
pd.DataFrame(Hrent).to_excel(Filename, sheet_name='Hrent', index=False)
T_JobOppLatCat.to_excel(Filename, sheet_name='JobOpp_LabCat')
T_Tran.to_excel(Filename, sheet_name='ACD_ACT') #drop index, do this later

Filename.save()

In [12]:
T_EREW

ER                    EW          
     Column_A  Column_B    Column_A  Column_B
0  103.672466  0.345575   89.494273  0.298314
1   92.655067  0.308850  121.011454  0.403372
2  103.672466  0.345575   89.494273  0.298314

In [13]:
pd.DataFrame(Hrent)

,0
0,208.035876
1,185.927687
2,208.035876


In [14]:
# # check carlibration 
# sio.loadmat('ZAT(Python).mat')['ZAttrI']

In [15]:
# # compare results generated from Matlab
# matZAT = sio.loadmat('Simplified_Matlab_Model_v3_Calibration/ZAT.mat')['ZAT']
# ZAT = matZAT[0,0]    # ZAT.dtype
# ZAttrI = np.moveaxis(ZAT['ZAttrI'], -1, 0)
# ZAttrIJ = np.moveaxis(ZAT['ZAttrIJ'], -1, 0)

# ZAttrI

In [16]:
# ### print dependences

# %load_ext watermark

# #show version, machine, and package information  # to check what packages we used: %watermark --iversions
# %watermark -v -m -p scipy,numpy,pandas,watermark,openpyxl,time 

# # date -  u:"Last updated, n:day and month names, t:time, z:zone
# %watermark -u -n -t -z  